In [ ]:
import os

import pandas as pd
import mlflow
import mlflow.tensorflow
from tensorflow.python.saved_model import tag_constants

from art.attacks.evasion import FastGradientMethod

from aidefender.exp.datasets import create_dataset

In [ ]:
models_dir = '/datadrive/alromano/aidefender/artifacts/models/'
tensorflow_savedmodel_dir = os.path.join(models_dir, 'customvision_cats_and_dogs_tf_savedmodel')
mlflow_dir = os.path.join(models_dir, 'mlflow_cats_and_dogs_customvision_tf/')

dataset_class = 'aidefender.exp.datasets.CatsAndDogsDataset'
data_path = '../artifacts/data/cats_dogs_small/'

In [ ]:
dataset = create_dataset(dataset_class, data_path)

In [ ]:
tf_meta_graph_tags=[tag_constants.SERVING]
tf_signature_def_key='serving_default'

In [ ]:
mlflow.tensorflow.save_model(
    tf_saved_model_dir=tensorflow_savedmodel_dir, 
    tf_meta_graph_tags=tf_meta_graph_tags, 
    tf_signature_def_key=tf_signature_def_key, 
    path=mlflow_dir,
    signature=dataset.model_signature
)

# Test the model

In [ ]:
mlflow_model = mlflow.pyfunc.load_model(mlflow_dir)

In [ ]:
mlflow_model.predict({'inputs': dataset.images[0:1]})

In [ ]:
type(mlflow_model._model_impl)

In [ ]:
next(iter(mlflow_model._model_impl.input_tensor_mapping.values()))

In [ ]:
next(iter(mlflow_model._model_impl.output_tensors.values()))

# Create ART classifier

In [ ]:
from aidefender.utils.mlflow import create_art_model

In [ ]:
art_model = create_art_model(mlflow_model)

In [ ]:
art_model.predict(dataset.images, batch_size=1).argmax(-1)

## Attack

In [ ]:
attack = FastGradientMethod(art_model)
attack.set_params(batch_size=1)

In [ ]:
images = dataset.images[0:3]

In [ ]:
images.shape

In [ ]:
images_adv = attack.generate(images)

In [ ]:
art_model.predict(images).argmax(-1)

In [ ]:
art_model.predict(images_adv).argmax(-1)